# flowField class for iterative solution

## Contents
- Subclass of numpy.ndarray()

### Basic flow descriptors
Define as a dictionary:
* $ \alpha, \beta $  describing the periodicity, and $Re$
* K,L,M,N describing the resolution:
    - K: No. of temporal frequencies
    - L: No. of positive streamwise Fourier modes
    - M: No. of positive spanwise Fourier modes
    - N: No. of Chebyshev collocation nodes in wall-normal 
    - The total size of each variable would be (2L+1)(2M+1)N
* Flow type: Couette or Poiseuille (use flags)- default to Couette
    - Boundary conditions on $u_0$
    - dPdx
    
### Values
* Values (This shouldn't be directly accessible- use reshapers to access values)
    - Store only fluctuations
    - Define a separate $u_{base}$


### Methods to be defined
Should I write these as a view() or simply as a reshape()?
* Viewers: 
    - 1D array for state
    - 4D array (streamwise, spanwise, variable ID, wall-normal)
* Truncating:
    - Returning only (u,w)
    - Returning only interior nodes, ignoring the walls
* Dot product
* Norm
* spectral2physical:
    - Do not change the flowField class itself, instead, write a new object
    
## Inputs
* Flow descriptors (see above)
    - Keep $ \alpha, \beta, Re$ as optional parameters, default values being those used in ChannelFlow: (1.14,2.5,400)
    * Resolution: L,M,N    
* Base flow type (flag):
    - Either linear, or quadratic. Assume linear as default. 
* Read from file- optional (flag):
    - Filename, format ('mat', 'npy', 'asc',...)
* Noise levels (default to zero):
    - Add random noise to the state vector whose norm is that as input.

## Tests for class
* Ensure that solving for flat-walled Couette and Poiseuille flows using direct inversion works
* Ensure that the above cases work with iterative solver. 

### Other notes to self:
* Write doc-strings for all the methods and inputs
* When initializing flowField as white-noise, think about smoothening the noise. 


## Collaborator comments
Use this cell for comments

---------------------------------------------------

## Inheriting numpy.ndarray()

Reference: http://docs.scipy.org/doc/numpy/user/basics.subclassing.html

In [1]:
import numpy as np

class C(np.ndarray): pass

arr = np.zeros(3,)
c_arr = arr.view(C)

print(type(arr))
print(type(c_arr))
print(c_arr)



<class 'numpy.ndarray'>
<class '__main__.C'>
[ 0.  0.  0.]


In [3]:
v= c_arr[:]
print(type(v))

v is c_arr


<class '__main__.C'>


False

In [5]:
print(type(v))

<class '__main__.C'>


In [6]:
print dir()

SyntaxError: invalid syntax (<ipython-input-6-633e6c7260eb>, line 1)

In [7]:
print(dir())

['C', 'In', 'Out', '_', '_1', '_2', '_4', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_ih', '_ii', '_iii', '_oh', '_sh', 'arr', 'c_arr', 'exit', 'get_ipython', 'np', 'quit', 'v']


In [8]:
print(c_arr)
v[1]=5.
print(c_arr)

[ 0.  0.  0.]
[ 0.  5.  0.]


In [9]:
v == c_arr

C([ True,  True,  True], dtype=bool)

Need to be careful when using `is`. It refers to the IDs of objects. If using arrays or elements of arrays, using `is` is a bad idea. Stick to `==` when comparing numerical values- which is usually all I care for.

In [2]:
gen = np.zeros(3,)
v1 = gen[:]
print(v1 is gen)
v1[1]=3.
print(gen)
print(gen[:] is gen)
print(id(gen))
print(id(gen[:]))
v2 = gen
print (v2 is gen)
print (id(v2))
print(v1 == gen)
print (v1[0] is gen[0])

False
[ 0.  3.  0.]
False
36104032
36568608
True
36104032
[ True  True  True]
False


## Reading files into dictionary


In [2]:
ls


flowConfig.txt  flowField Class.ipynb


In [3]:
%cat flowConfig.txt

alpha   1.14	% (float) Streamwise wavenumber of periodic box
beta    2.5	% (float) Spanwise wavenumber of periodic box
omega   0.0	% (float) Fundamental frequency resolved in computation
ReLam   400.0 	% (float) Re based on laminar base flow
isPois  0	% (0 or 1) 1 for Poiseuille flow, 0 for Couette
noise   0.0	% (float) Norm of white noise to be added
L       23      % (int) Number of positive streamwise Fourier modes 
M       23	% (int) Number of positive spanwise Fourier modes
N       35	% (int) Number of Chebyshev collocation nodes
K       0	% (int) Number of positive Fourier frequency modes
--- ---- ---- WARNING: No text goes after this line


In [58]:
flowDict = {}
with open("flowConfig.txt",'r') as f:
    for line in f:
        (key,val) = line.split()[:2]
        flowDict[key] = float(val)


In [59]:
defaultDict = {'alpha':1.14, 'beta' : 2.5, 'omega':0.0, 'L': 0.0, 'M': 0.0, 'N': 0.0, 'K':0.0,
               'ReLam': 400.0, 'isPois':0.0, 'noise':0.0 , 'testvar':5.5}
for key in defaultDict:
    if key not in flowDict:
        flowDict[key] = defaultDict[key]
        print(key,flowDict[key],type(flowDict[key]))

testvar 5.5 <class 'float'>


In [50]:
print(flowDict)
print( 2.5+flowDict['alpha'])

{'beta': 2.5, 'noise': 0.0, 'ReLam': 400.0, 'omega': 0.0, 'N': 35.0, 'K': 0.0, 'L': 23.0, 'alpha': 1.14, 'M': 23.0, '---': 0.0, 'isPois': 0.0}
3.6399999999999997


In [45]:
flowDict['alpha'],flowDict['beta']
print (2+float(flowDict['alpha']))
print(flowDict.values)
float(flowDict['beta'])

flowDict['alpha'] = float(flowDict['alpha'])
flowDict['alpha']+2.5

3.1399999999999997
<built-in method values of dict object at 0x7fe3a6c14808>


KeyError: 'zeta'

In [13]:
import numpy as np
arr = np.ones(3)
print(arr)

arr1 = 5.*np.ones((3,4))
print(arr1)

print(arr*arr1)

[ 1.  1.  1.]
[[ 5.  5.  5.  5.]
 [ 5.  5.  5.  5.]
 [ 5.  5.  5.  5.]]


ValueError: operands could not be broadcast together with shapes (3,) (3,4) 

In [10]:
import numpy as np

np.arange(1,15)

for k in range(1,5):
    print (k)

1
2
3
4


## Defining flowField class that inherits np.ndarray

For starters, defining the class to initialize an empty ndarray along with a dictionary provided during initialization.

I need to verify that the dictionary being supplied has all the info I need to go with a flowField class. But, I can't verify this all the time. This is how I'll deal with it: 
* Verify the dictionary only when constructing a flowField class. So, that's when
    - Explicitly constructing an instance of the class
    - Viewing a given array as an instance of the class
* When slicing an array of the flowField class (to obtain another instance of the class), don't bother with the check. 

Basically, ensure every instance of the flowField class has a valid dictionary, and then stop worrying about it. 

So, we need a function that verifies the validity of dictionary, or creates one if one isn't supplied. The parameters required in the dictionary, and their defaults, are as follows:
* alpha : 1.14    
    *Wavenumber of fundamental streamwise Fourier mode*
* beta :  2.5    
    *Wavenumber of fundamental spanwise Fourier mode*
* omega: 0.0    
    *Fundamental frequency*
* ReLam: 400.0  
    *Reynolds number of the laminar base flow*
* isPois: 0     
    *Flag for base flow type. 0: Couette, 1: Poiseuille*
* noise: 0.0    
    *Norm of noise to be added to the flow*
* L: 0          
    *Number of (positive) harmonics of fundamental streamwise Fourier mode.*
* M: 0          
    *Number of (positive) harmonics of fundamental spanwise Fourier mode*
* N: 35         
    *Number of Chebyshev collocation nodes*
* K: 0         
    *Number of (positive) harmonics of fundamental frequency*

** For now, I'm ignoring the fact that np.ndarray can be viewed as a subclass. I'm defining the dictionary and checks in flowField.__new__(). Later, I'll have to move this to __array_finalize__ so that the dictionary is defined for cases when either an explicit constructor call is made or a view-casting is done**

### Ensure these features:
* View-casting:
    - Take an existing numpy array and view it as a flowField class, given that a flowDict dictionary is supplied.
* Explicit construction:
    - Construct a randomField object based on noise levels supplied
* New from template:
    - Truncate and expand an instance of flowField to obtain a new instance with a changed dictionary (to reflect the truncation of L,M,N, or K)



** I have three cases to consider when building a flowField instance. Suppose L = 1**:
- The flowField has streamwise wavenumbers $-\alpha, 0, \alpha$
- The flowField only has streamwise wavenumbers $-\alpha, \alpha$
- The flowField only has streamwise wavenumber $\alpha$
    
This question becomes particularly important if only a half-plane or half-volume of the wavenumber space is considered. 

This is how I chose to resolve this issue. Considering L:
* If L = 0, then the flowField is of wavenumber $\alpha$
* If L = n ($\in \mathbb{N}$), then the flowField is resolved in wavenumbers {$0,\alpha,2\alpha,..,n\alpha$}.
* If L = -n ($n \in \mathbb{N}$), then the flowField is resolved in wavenumbers {$-n\alpha, (-n+1)\alpha,..,0,\alpha,...,n\alpha$}

This way, a flowField can be defined in three different ways for each of the three Fourier axes (streamwise, spanwise, temporal):
* As just one Fourier mode, $\alpha$ (which could be positive, negative, or zero). 
* As a collection of `n` harmonics (positive integer multiples of a positive/negative wavenumber), along with mode zero (the invariant mode).
* As a collection of `2n+1` harmonics, from $-n\alpha$ through $n\alpha$.


I think this should be enough to deal with most cases. We will later need to define collections of flowFields. 

In [72]:
# In case dictionary is 'None':
defaultDict = {'alpha':1.14, 'beta' : 2.5, 'omega':0.0, 'L': 0.0, 'M': 0.0, 'N': 0.0, 'K':0.0,
               'ReLam': 400.0, 'isPois':0.0, 'noise':0.0 }

def verify_dict(tempDict):
    if tempDict is None:
        tempDict = defaultDict
    else: 
        for key in defaultDict:
            if key not in tempDict:
                tempDict[key] = defaultDict[key]
    return tempDict

flowDict = None
flowDict = verify_dict(flowDict)
print(flowDict)

{'beta': 2.5, 'isPois': 0.0, 'L': 0.0, 'omega': 0.0, 'N': 0.0, 'K': 0.0, 'alpha': 1.14, 'M': 0.0, 'noise': 0.0, 'ReLam': 400.0}


In [70]:
import numpy as np

defaultDict = {'alpha':1.14, 'beta' : 2.5, 'omega':0.0, 'L': 0.0, 'M': 0.0, 'N': 0.0, 'K':0.0,
               'ReLam': 400.0, 'isPois':0.0, 'noise':0.0 }

def verify_dict(tempDict):
    if tempDict is None:
        tempDict = defaultDict
    else: 
        for key in defaultDict:
            if key not in tempDict:
                tempDict[key] = defaultDict[key]
    return tempDict

def read_dictFile(dictFile):
    tempDict = {}
    with open("flowConfig.txt",'r') as f:
        for line in f:
            (key,val) = line.split()[:2]
            tempDict[key] = float(val)    
    return tempDict

class flowField(np.ndarray):
    
    
    def __new__(cls, flowDict=None, dictFile= None):
        # The shape of the numpy array comes from the dictionary
        #   so it is absolutely necessary to have the dictionary
        if flowDict is None:
            if dictFile is not None:
                flowDict = verify_dict(read_dictFile(dictFile))
            else:
                flowDict = default_dict
        else:
            flowDict = verify_dict(flowDict)
            
        L = int(flowDict['L'])
        M = int(flowDict['M'])
        N = int(flowDict['N'])
        K = int(flowDict['K'])
        
        obj = np.ndarray.__new__(cls, shape=(L,M,K,N),dtype=float)
        return obj

In [4]:
cat flowConfig.txt

alpha   1.14	% (float) Streamwise wavenumber of periodic box
beta    2.5	% (float) Spanwise wavenumber of periodic box
omega   0.0	% (float) Fundamental frequency resolved in computation
ReLam   400.0 	% (float) Re based on laminar base flow
isPois  0	% (0 or 1) 1 for Poiseuille flow, 0 for Couette
noise   0.0	% (float) Norm of white noise to be added
L       23      % (int) Number of positive streamwise Fourier modes 
M       23	% (int) Number of positive spanwise Fourier modes
N       35	% (int) Number of Chebyshev collocation nodes
K       0	% (int) Number of positive Fourier frequency modes
--- ---- ---- WARNING: No text goes after this line
